Question 4

In [2]:
import numpy as np
import math

spk_mat = np.load("spk_mat.npy")  # shape is (8, time_points)
entropies = []

#each neuron(row)
for i in range(8):
    neuron_data = spk_mat[i]
    p = np.mean(neuron_data)  # probability of spike (value 1)
    if p == 0 or p == 1:
        h = 0  # no uncertainty
    else:
        h = -p * math.log2(p) - (1 - p) * math.log2(1 - p)
    entropies.append(h)
    print(f"Neuron {i+1} entropy: {h:.4f} bits")


Neuron 1 entropy: 0.9921 bits
Neuron 2 entropy: 0.9396 bits
Neuron 3 entropy: 0.9128 bits
Neuron 4 entropy: 0.8633 bits
Neuron 5 entropy: 0.8521 bits
Neuron 6 entropy: 0.8293 bits
Neuron 7 entropy: 0.8700 bits
Neuron 8 entropy: 0.8796 bits


In [ ]:
#B
#For each time point, we sum across the 8 neurons → values range from 0 to 8.

# Sum activity across neurons at each time point
network_activity = np.sum(spk_mat, axis=0)  # shape: (time_points,)

# Count how often each sum value (0 to 8) appears
activity_values, counts = np.unique(network_activity, return_counts=True)

# Convert counts to probabilities
probabilities = counts / np.sum(counts)

# Calculate entropy
network_entropy = 0
for p in probabilities:
    if p > 0:
        network_entropy += -p * math.log2(p)

print(f"Network entropy: {network_entropy:.4f} bits")


Network entropy: 2.8774 bits


In [4]:
# C 
# Mutual information between two binary vectors
def mutual_info(x, y):
    mi = 0
    for a in [0, 1]:
        for b in [0, 1]:
            # Joint probability
            p_ab = np.mean((x == a) & (y == b))
            # Marginals
            p_a = np.mean(x == a)
            p_b = np.mean(y == b)
            if p_ab > 0:
                mi += p_ab * math.log2(p_ab / (p_a * p_b))
    return mi

# Target neurons: 3, 7, 8 (index 2, 6, 7)
targets = [2, 6, 7]

for target in targets:
    print(f"\nNeuron {target + 1}:")
    best_mi = 0
    best_neuron = None
    for other in range(8):
        if other == target:
            continue
        mi = mutual_info(spk_mat[target], spk_mat[other])
        print(f"  Mutual info with neuron {other + 1}: {mi:.4f} bits")
        if mi > best_mi:
            best_mi = mi
            best_neuron = other + 1
    print(f"→ Best predictor for neuron {target + 1} is neuron {best_neuron} with MI = {best_mi:.4f} bits")


Neuron 3:
  Mutual info with neuron 1: 0.4413 bits
  Mutual info with neuron 2: 0.2643 bits
  Mutual info with neuron 4: 0.0949 bits
  Mutual info with neuron 5: 0.0553 bits
  Mutual info with neuron 6: 0.0328 bits
  Mutual info with neuron 7: 0.0193 bits
  Mutual info with neuron 8: 0.0244 bits
→ Best predictor for neuron 3 is neuron 1 with MI = 0.4413 bits

Neuron 7:
  Mutual info with neuron 1: 0.0392 bits
  Mutual info with neuron 2: 0.0263 bits
  Mutual info with neuron 3: 0.0193 bits
  Mutual info with neuron 4: 0.0105 bits
  Mutual info with neuron 5: 0.2576 bits
  Mutual info with neuron 6: 0.1523 bits
  Mutual info with neuron 8: 0.0032 bits
→ Best predictor for neuron 7 is neuron 5 with MI = 0.2576 bits

Neuron 8:
  Mutual info with neuron 1: 0.0475 bits
  Mutual info with neuron 2: 0.0317 bits
  Mutual info with neuron 3: 0.0244 bits
  Mutual info with neuron 4: 0.1787 bits
  Mutual info with neuron 5: 0.0074 bits
  Mutual info with neuron 6: 0.0045 bits
  Mutual info with 